In [14]:
Symbol = 'BTCUSDT'
end = datetime.now(timezone.utc)
start = end - timedelta(days=6)

#0.Import Libraries
import pandas as pd
import numpy as np
from datetime import datetime, timezone, timedelta
import time
from pybit.unified_trading import HTTP
from dotenv import load_dotenv
import os
import hmac, hashlib, urllib.parse, requests, math, threading, json

#1.Setup
load_dotenv('keys.env')
bybit_key = os.getenv('BYBIT_API_KEY')
bybit_secret = os.getenv('BYBIT_API_SECRET')
bybit_testnet = os.getenv('BYBIT_TESTNET')
session = HTTP(testnet = False, api_key = bybit_key, api_secret = bybit_secret)

Aster_key = os.getenv('ASTER_API_KEY')
Aster_secret = os.getenv('ASTER_API_SECRET')
RECV_WINDOW    = 50000
USER_AGENT     = "Tradeloggetter/1.0"

#2.Get Bybit
def to_ms(dt: datetime) -> int:
    # dt は timezone-aware（UTC推奨）
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    return int(dt.timestamp() * 1000)

def fetch_all_closed_pnl(session, category: str, symbol: str | None,
                         start: datetime | None, end: datetime | None,
                         limit: int = 200, sleep_sec: float = 0.1) -> pd.DataFrame:
    """
    Bybit v5 /v5/position/closed-pnl をカーソルで最後まで取得して DataFrame で返す
    """
    params = {
        "category": category,          # "linear" か "inverse"
        "limit": min(max(limit,1),200)
    }
    if symbol:
        params["symbol"] = symbol
    if start:
        params["startTime"] = to_ms(start)
    if end:
        params["endTime"] = to_ms(end)

    all_rows = []
    cursor = None
    while True:
        if cursor:
            params["cursor"] = cursor
        resp = session.get_closed_pnl(**params)
        # retCode チェック
        if resp.get("retCode") != 0:
            raise RuntimeError(f"Bybit API error: {resp.get('retCode')} {resp.get('retMsg')}")
        result = resp.get("result", {})
        rows = result.get("list", []) or []
        all_rows.extend(rows)
        cursor = result.get("nextPageCursor")
        if not cursor:
            break
        time.sleep(sleep_sec)  # Rate limit 余裕

    # DataFrame 化（必要に応じて列を整形）
    df = pd.DataFrame(all_rows)
    # 代表的な時刻列（ドキュメント上 UTA は createdTime / Classic は updatedTime）
    for col in ("createdTime", "updatedTime", "execTime"):
        if col in df.columns:
            df[col] = pd.to_datetime(df[col].astype("int64"), unit="ms", utc=True)
    return df

#3.Get Aster
# ==== Aster REST 基本情報 ====
ASTER_BASE    = "https://fapi.asterdex.com"
ASTER_KEY_HDR = "X-MBX-APIKEY"   # AsterはBinance互換ヘッダ

# ==== 読み込んだキーを使用 ====
ASTER_API_KEY    = Aster_key
ASTER_API_SECRET = Aster_secret

RECV_WINDOW = 50000
USER_AGENT  = "TradeLogGetter/1.0"

def to_ms(dt: datetime) -> int:
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    return int(dt.timestamp() * 1000)

def _now_ms() -> int:
    return int(time.time() * 1000)

def _hmac_sha256(secret: str, msg: str) -> str:
    return hmac.new(secret.encode("utf-8"), msg.encode("utf-8"), hashlib.sha256).hexdigest()

def _request(method: str, path: str, params: dict | None = None) -> dict:
    """SIGNED/USER_DATA/TRADE 共通のHTTP呼び出し（QueryString署名）"""
    url = ASTER_BASE + path
    q = dict(params or {})
    q.setdefault("recvWindow", RECV_WINDOW)
    q["timestamp"] = _now_ms()
    qs = urllib.parse.urlencode(q, doseq=True)
    sig = _hmac_sha256(ASTER_API_SECRET, qs)

    headers = {ASTER_KEY_HDR: ASTER_API_KEY, "User-Agent": USER_AGENT}

    if method.upper() == "GET":
        full_url = f"{url}?{qs}&signature={sig}"
        r = requests.get(full_url, headers=headers, timeout=30)
    elif method.upper() == "POST":
        headers["Content-Type"] = "application/x-www-form-urlencoded"
        body = f"{qs}&signature={sig}"
        r = requests.post(url, data=body, headers=headers, timeout=30)
    elif method.upper() == "DELETE":
        headers["Content-Type"] = "application/x-www-form-urlencoded"
        body = f"{qs}&signature={sig}"
        r = requests.delete(url, data=body, headers=headers, timeout=30)
    else:
        raise ValueError("Unsupported method")

    if r.status_code >= 400:
        raise RuntimeError(f"Aster {method} {path} failed: {r.status_code} {r.text[:500]}")
    return r.json()

def fetch_account_trade_list_aster(symbol: str,
                                   start: datetime,
                                   end: datetime,
                                   limit: int = 1000,
                                   sleep_sec: float = 0.15) -> pd.DataFrame:
    """
    GET /fapi/v1/userTrades を 7日スライスで全件取得して DataFrame で返す。
    - start/end: timezone-aware datetime（UTC推奨）
    - 1リクエスト最大1000件、期間は最大7日（公式仕様）
    - 期間内に1000件超がある場合は「最後の time + 1ms」で次リクエスト（fromIdはstart/endと併用不可のため不使用）
    """
    start_ms_total = to_ms(start)
    end_ms_total   = to_ms(end)

    all_rows = []
    window_start_ms = start_ms_total
    seven_days_ms = 7 * 24 * 60 * 60 * 1000

    while window_start_ms <= end_ms_total:
        window_end_ms = min(window_start_ms + seven_days_ms - 1, end_ms_total)
        cursor_ms = window_start_ms

        while True:
            params = {
                "symbol": symbol,
                "startTime": cursor_ms,
                "endTime": window_end_ms,
                "limit": min(max(limit, 1), 1000)
            }
            resp = _request("GET", "/fapi/v1/userTrades", params)
            rows = resp or []
            if not isinstance(rows, list):
                # 互換系でオブジェクト返す場合に備えて補正
                rows = rows.get("list", rows.get("data", [])) or []

            if not rows:
                break

            all_rows.extend(rows)

            # 期間内に limit 到達の可能性 → 最後の time を見て 1ms 進めて続行
            if len(rows) >= params["limit"]:
                last_time = int(rows[-1]["time"])
                next_ms = last_time + 1
                if next_ms <= window_end_ms:
                    cursor_ms = next_ms
                    time.sleep(sleep_sec)
                    continue

            break  # 取り切った

        # 次の7日ウィンドウへ
        window_start_ms = window_end_ms + 1
        time.sleep(sleep_sec)

    df = pd.DataFrame(all_rows)
    if not df.empty and "time" in df.columns:
        df["time"] = pd.to_datetime(df["time"].astype("int64"), unit="ms", utc=True)
    return df

In [21]:
df_pnl = fetch_all_closed_pnl(session=session, category="linear", symbol=Symbol, start=start, end=end, limit=200)
# 数値列を float に変換
for col in ["closedPnl", "openFee", "closeFee"]:
    df_pnl[col] = pd.to_numeric(df_pnl[col], errors="coerce")

# 日付抽出
df_pnl["updatedTime"] = pd.to_datetime(df_pnl["updatedTime"], utc=True)
df_pnl["date"] = df_pnl["updatedTime"].dt.date

# 日毎に合計
df_daily = (
    df_pnl
    .groupby("date")[["closedPnl", "openFee", "closeFee"]]
    .sum()
    .reset_index()
)
df_daily

,date,closedPnl,openFee,closeFee
0,2025-09-23,-6.362615,3.378088,3.381928
1,2025-09-24,-24.674834,10.109333,10.106891
2,2025-09-25,-32.721992,14.605362,14.603430
3,2025-09-26,-21.239415,11.613966,11.616788


In [24]:
# 直近7日（UTC）の BTCUSDT の約定履歴
df_trades = fetch_account_trade_list_aster(Symbol, start, end)
# time を datetime に変換して日付列を作る
df_trades["time"] = pd.to_datetime(df_trades["time"], utc=True)
df_trades["date"] = df_trades["time"].dt.date

# 数値列を float に変換（文字列になっている可能性があるため）
df_trades["realizedPnl"] = pd.to_numeric(df_trades["realizedPnl"], errors="coerce")
df_trades["commission"] = pd.to_numeric(df_trades["commission"], errors="coerce")

# 日毎に集計
df_daily = (
    df_trades
    .groupby("date")[["realizedPnl", "commission"]]
    .sum()
    .reset_index()
)

df_daily

,date,realizedPnl,commission
0,2025-09-23,-7.4988,2.155193
1,2025-09-24,3.1634,16.987355
2,2025-09-25,12.9474,38.795065
3,2025-09-26,-1.0532,15.897048


In [25]:
df_pnl

,symbol,orderType,leverage,updatedTime,side,orderId,closedPnl,openFee,closeFee,avgEntryPrice,qty,cumEntryValue,createdTime,orderPrice,closedSize,avgExitPrice,execType,fillCount,cumExitValue,date
0,BTCUSDT,Market,10,2025-09-26 03:10:17.020000+00:00,Sell,ae4bc8d2-4695-488c-8ebc-1e07e12de705,-3.030058,0.602531,0.601527,109551.1,0.01,1095.511,2025-09-26 03:10:17.017000+00:00,108275.2,0.01,109368.5,Trade,4,1093.685,2025-09-26
1,BTCUSDT,Market,10,2025-09-26 03:00:14.106000+00:00,Sell,2ab18227-6118-4b83-97d4-ffe7a311d33e,-2.490975,0.603341,0.602634,109698.3,0.01,1096.983,2025-09-26 03:00:14.104000+00:00,108480.6,0.01,109569.8,Trade,1,1095.698,2025-09-26
2,BTCUSDT,Market,10,2025-09-26 02:50:11.123000+00:00,Sell,36a9f945-a4ea-4f30-b38c-c18b7286c56e,0.123164,0.602553,0.603283,109555,0.01,1095.55,2025-09-26 02:50:11.120000+00:00,108591.2,0.01,109687.9,Trade,2,1096.879,2025-09-26
3,BTCUSDT,Market,10,2025-09-26 02:40:08.248000+00:00,Sell,22da8774-405c-455d-acc8-9aa26bee0d81,-0.911943,0.602391,0.602552,109525.6,0.01,1095.256,2025-09-26 02:40:08.246000+00:00,108459.4,0.01,109554.9,Trade,1,1095.549,2025-09-26
4,BTCUSDT,Market,10,2025-09-26 02:30:03.951000+00:00,Sell,a8c9a321-98a7-47a0-9442-e1591fbc2c3e,-2.565529,0.603138,0.602390,109661.5,0.01,1096.615,2025-09-26 02:30:03.949000+00:00,108431.5,0.01,109525.5,Trade,6,1095.255,2025-09-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,BTCUSDT,Market,10,2025-09-24 05:10:33.740000+00:00,Sell,b8f24012-0749-4bca-a12b-5ea8a41ff73b,-2.575331,0.987031,0.986700,112162.6,0.016,1794.6016,2025-09-24 05:10:33.738000+00:00,111013.2,0.016,112125,Trade,1,1794,2025-09-24
60,BTCUSDT,Market,10,2025-09-24 04:19:46.153000+00:00,Sell,23bb5921-0669-4672-95e7-8266a29703ce,-16.208605,1.171199,1.163539,112076.4211,0.019,2129.452,2025-09-24 04:19:46.151000+00:00,110230,0.019,111343.4,Trade,1,2115.5246,2025-09-24
61,BTCUSDT,Market,10,2025-09-23 21:36:43.848000+00:00,Sell,70711d60-5140-4a7d-a9b9-46949f1e7fbe,6.133889,1.104960,1.109551,111612.1,0.018,2009.0178,2025-09-23 21:36:43.846000+00:00,110955.3,0.018,112075.9,Trade,1,2017.3662,2025-09-23
62,BTCUSDT,Market,10,2025-09-23 20:06:41.964000+00:00,Buy,a5f52fe9-50dc-4dc1-8a39-c692069e0da2,-5.503435,1.104712,1.106523,111587.1,0.018,2008.5678,2025-09-23 20:06:41.962000+00:00,112887.7,0.018,111770,Trade,1,2011.86,2025-09-23


In [29]:
1093.685 - 1095.511 - 0.60253105 - 0.60152676
# Bybitの損益は手数料控除後

-3.030057810000022